In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

from analyzer.metric import MetricCalculator
from data import Reader, Saver
from plot import Drawer
from analyzer.stat_criterion.stat_criteria import StatCriteria
from encoder import Encoder
from preparation import DataPreparation, Normalizer

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.regression.linear_model import OLS
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures

In [2]:
# читаем датасет в память из внешнего источника
df_original = Reader.read_csv('https://raw.githubusercontent.com/Semendyeav/datasets/refs/heads/main/PDA120_KNN_IncomeMoney.csv')
df = df_original.copy().reset_index(drop=True)
df.head(10)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,33,Federal-gov,331615,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,1848,40,United-States,>50K
1,51,Private,173987,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,>50K
2,31,Private,44464,Bachelors,13,Never-married,Exec-managerial,Not-in-family,White,Female,0,1564,60,United-States,>50K
3,50,Private,263200,5th-6th,3,Married-spouse-absent,Other-service,Unmarried,White,Female,0,0,34,Mexico,<=50K
4,39,Private,175232,Some-college,10,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
5,59,Private,43221,9th,5,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,60,United-States,>50K
6,42,Self-emp-not-inc,54651,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,Cuba,>50K
7,27,Self-emp-not-inc,87745,Assoc-voc,11,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,70,United-States,<=50K
8,31,Private,34572,Bachelors,13,Married-civ-spouse,Sales,Husband,White,Male,0,0,55,United-States,<=50K
9,60,Private,250552,HS-grad,9,Married-civ-spouse,Other-service,Husband,White,Male,0,0,40,United-States,>50K


fnlwgt (final weight) – примерная оценка количества людей, которое представляет каждая строка данных

educational-num – длительность обучения

capital gain - это экономическая концепция, определяемая как прибыль, полученная от продажи актива, стоимость которого увеличилась за период владения.

capital loss — это убыток, который возникает, когда стоимость капитальных активов, таких как инвестиции или недвижимость, снижается.

## Постановка задачи
1. Определить какие признаки более всего влияют на таргет, 
2. построить модель наивного Байеса когда все колонки переведны в вид категорий, 
3. построить модель логистической регрессии с L1 регуляризацией, чтобы выявить какие признаки "мусорные", 
4. после лекции в понедельник построить дерево решений и модель svm.